In [1]:
import pickle, json, requests, csv, copy, os, re, sys, math, random
import numpy as np
import pprint as pp
import urllib.request, urllib.parse
from sklearn.metrics.pairwise import cosine_similarity
from json import JSONDecodeError
from text_to_uri import *

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

In [7]:
classes = [x[1] for x in os.walk('../data/review/KDD2014-Chen-Dataset/Non-Electronics')][0]

In [62]:
classes = ['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']

In [ ]:
class_uri = ['/c/en/atheism',
            '/c/en/graphics',
            '/c/en/operating_system',
            '/c/en/ibm',
            '/c/en/mac',
            '/c/en/windows',
            '/c/en/sale',
            '/c/en/auto',
            '/c/en/motorcycle',
            '/c/en/baseball',
            '/c/en/hockey',
            '/c/en/crypt',
            '/c/en/electronics',
            '/c/en/medical',
            '/c/en/space',
            '/c/en/christian',
            '/c/en/gun',
            '/c/en/middle_east',
            '/c/en/politics',
            '/c/en/religion']

In [15]:
PATHS_FROM = pickle.load(open("../wordEmbeddings/PATHS_FROM_01.pickle", "rb"))

AttributeError: Can't get attribute 'Path' on <module '__main__'>

In [2]:
KG_VECTORS_2 = pickle.load(open("../wordEmbeddings/KG_VECTORS_2.pickle", "rb"))

In [3]:
for key in KG_VECTORS_2:
    print(key, len(KG_VECTORS_2[key]))

/c/en/company 21267
/c/en/education 7514
/c/en/artist 7035
/c/en/athlete 5156
/c/en/officer 12797
/c/en/transport 14151
/c/en/building 34099
/c/en/nature 24775
/c/en/village 11549
/c/en/animal 62420
/c/en/plant 46970
/c/en/album 5381
/c/en/film 15207
/c/en/writing 18824


In [2]:
data = np.load('../results/test_5.npz')

In [3]:
print(data.keys())

['align_unseen', 'gt_seen', 'pred_unseen', 'align_seen', 'pred_seen', 'gt_unseen']


In [5]:
print(data['gt_unseen'], data['pred_unseen'], data['align_unseen'])

[[0. 0. 0. ... 1. 0. 0.]
 [0. 0. 0. ... 1. 0. 0.]
 [0. 0. 0. ... 1. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 1.]] [[0. 0. 1. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 1. 0.]] [[[2.47972156e-03 2.73659639e-03 2.02324498e-03 ... 3.87769453e-02
   9.77866575e-02 9.69534069e-02]
  [9.36909858e-03 9.62136686e-03 7.42427306e-03 ... 2.30694991e-02
   2.38265380e-01 3.20462227e-01]
  [2.78055595e-05 3.19405553e-05 2.46378622e-05 ... 4.07097396e-04
   5.02543598e-02 9.46804345e-01]
  ...
  [1.32638263e-03 1.42502296e-03 1.00284070e-03 ... 2.21102890e-02
   7.81787410e-02 7.99784884e-02]
  [5.52700879e-03 5.93346125e-03 4.26079705e-03 ... 2.06842422e-02
   1.01126134e-01 1.16921619e-01]
  [6.46062428e-04 6.23823900e-04 4.90807986e-04 ... 2.92846859e-02
   1.86704919e-01 2.34603584e-01]]

 [[6.90063986e-04 9.84093989e-04 1.23169867e-03 ... 3.82701159e-02
   3.81

In [66]:
def process_text(filename):
    f = open(filename, 'r')
    lines = [line for line in f]
    f.close()
    
    processed = []
    for line in lines:
        if re.search(r'[a-zA-Z0-9]', line) is None:
            line = ''
        line = re.sub(r'([\w.-]+)@([\w.-]+)', 'EMAIL', line)
        line = re.sub(r'[^A-Za-z0-9_.\-\"\',()?\s]', ' ', line)
#         line = re.sub(r'\s+', ' ', line)
#         print(line)
        processed.append(line)
    ans = ''.join(processed)
    ans = re.sub(r'\s+', ' ', ans)
    return ans

In [68]:
print(process_text('..\\data\\newsgroup\\20news-18828\\talk.politics.misc\\176865'))

From EMAIL (Steve Hendricks) Subject Re Price Controls for Medical Care (WAS Re We're from the govt...) In article EMAIL EMAIL writes In article EMAIL , EMAIL (Steve Hendricks) writes Thanks to Kim for following up. I was hoping that someone would bring up the issue of cost compensation. The problem with the argument is that it fails to explain why kidney dialysis (RD) services have expanded massively in the last decade. After all, no one is forcing private providers to offer this service. If they are losing money on the treatment (which according to information I've collected from several providers they are not) why would they not simply limit their losses by cutting back on services (engage in effective rationing of supply) rather than expand the coverage to a larger market that must then be compensated by raising prices in other areas? Perhaps there is a competitive advantage in there after all. Would not reputable and dedicated physicians prefer to affiliate with an institution tha

In [67]:
for c in classes:
    directory = '..\\data\\newsgroup\\20news-18828-preprocessed\\'+c
    if not os.path.exists(directory):
        os.makedirs(directory)
    for root, dirs, files in os.walk('..\\data\\newsgroup\\20news-18828\\'+c):
        for f in files:
            text = process_text(os.path.join(root, f))
            with open(os.path.join(directory, f), 'w') as the_file:
                the_file.write(text)

In [4]:
data = json.load(open('..\\data\\review\\50EleReviews.json', 'r'))

In [10]:
print(len(data['X']), data['target_names'])

50000 ['Monitor', 'Watch', 'Blu Ray Player', 'CDPlayer', 'Network Adapter', 'Battery', 'Graphics Card', 'Xbox', 'Telephone', 'Car Stereo', 'Video Recorder', 'Alarm Clock', 'Camera', 'Camcorder', 'HardDrive', 'Home Theater System', 'Subwoofer', 'Rice Cooker', 'Webcam', 'Kindle', 'Scanner', 'Radar Detector', 'Vacuum', 'Cable Modem', 'CellPhone', 'Wireless Router', 'Printer', 'Fan', 'Amplifier', 'Microwave', 'GPS', 'Microphone', 'TV', 'Projector', 'Mouse', 'Voice Recorder', 'Memory Card', 'Iron', 'Laptop', 'Headphone', 'Video Player', 'MP3 Player', 'Keyboard', 'Tablet', 'Media Player', 'Lamp', 'Remote Control', 'Speaker', 'Computer', 'DVDPlayer']


In [2]:
class_uri = ['company',
            'education',
            'artist',
            'athlete',
            'officer',
            'transport',
            'building',
            'nature',
            'village',
            'animal',
            'plant',
            'album',
            'film',
            'writing']

In [7]:
f = open('../data/glove/glove.6B.200d.txt', 'r', encoding = 'utf8')
vec = np.zeros((14, 200))
for line in f:
    tokens = line.strip().split()
    if tokens[0] in class_uri:
        vec[class_uri.index(tokens[0])] = np.array(tokens[1:])

In [12]:
np.set_printoptions(threshold=np.nan)
print(vec)

[[ 1.1892e-01  1.5988e-01 -2.3317e-01 -9.1807e-02  4.7830e-01 -1.2479e-01
  -2.9638e-01  3.1465e-02  3.0812e-01 -8.4056e-02  5.1236e-01  6.9321e-02
  -8.5371e-02  1.4773e-01  6.0178e-01 -4.2442e-01 -5.2851e-02  3.8858e-01
   5.1832e-01 -1.7959e-01  5.2617e-02  2.6163e+00 -3.2562e-01 -2.2209e-01
   1.0511e-01  6.4953e-01 -6.1608e-01  3.7076e-01 -2.2254e-02 -2.3423e-01
  -8.0499e-02  1.8259e-02 -3.3002e-01 -5.3083e-01 -3.3396e-02 -3.9484e-01
  -5.0945e-01 -1.8411e-01 -6.0162e-01 -1.4134e-01 -5.0679e-01 -3.1507e-01
  -1.8976e-01 -3.9858e-01 -8.2417e-01 -4.4351e-01  3.0623e-01 -1.1439e-01
  -7.1624e-01  1.8007e-01  1.0228e-01  4.5982e-01 -2.4761e-01 -9.7277e-01
   3.9836e-01 -1.6986e-01 -9.7321e-01  3.6767e-01  2.2034e-01  3.7272e-01
   6.6823e-01  9.0374e-02 -4.2359e-01  1.1962e-01 -1.5005e-01 -2.4245e-01
  -7.2183e-01  6.6515e-01  9.6607e-04  3.5127e-01  6.1904e-01  6.8122e-02
   3.1885e-01  1.2140e-01 -1.4395e-01  1.3806e+00  4.5411e-01 -4.6329e-01
  -5.4043e-01  3.2039e-02  1.3447e-01 

In [15]:
corr = np.zeros((14,14))
for i in range(len(corr)):
    for j in range(len(corr)):
        corr[i,j] = 1 / cosine_similarity(vec[i].reshape(1, -1), vec[j].reshape(1, -1))
print(corr)

[[ 1.          3.48015041  3.4643388   9.40120014  2.19185038  2.74568754
   2.17394948  3.76938124  3.66029392  3.78955797  2.42288786  4.83377273
   2.79688931  3.27147818]
 [ 3.48015041  1.          4.52596798  4.04786421  3.15599342  2.84282284
   2.52547249  3.14469988  3.42228507  3.3308005   7.25743652  9.05226322
   3.85448512  2.35233899]
 [ 3.4643388   4.52596798  1.          2.87750146  4.57319505 44.84519418
   4.15086931  3.1947996   5.07387457  4.21762299  6.67299314  1.83102986
   2.24207568  2.27638461]
 [ 9.40120014  4.04786421  2.87750146  1.          4.29379827 11.89427691
   8.63831934  6.38999357  6.07202305  6.65325797 18.27515279  8.20317891
   4.95924802  5.28723084]
 [ 2.19185038  3.15599342  4.57319505  4.29379827  1.          3.13786161
   3.12366412  5.9950142   3.27549442  5.56487757  5.57148245 24.22866812
   4.38664166  4.80898454]
 [ 2.74568754  2.84282284 44.84519418 11.89427691  3.13786161  1.
   3.30058997  4.45896941  5.05803155  3.1231304   4.254392

In [21]:
for i in range(len(corr)):
    print(' '.join([str(x) for x in list(corr[i])]))

1.0000000000000002 3.4801504130242837 3.464338802507882 9.40120014368047 2.1918503774183296 2.745687544082795 2.1739494834933217 3.769381238149405 3.660293916814092 3.78955797331346 2.422887862365622 4.83377272778643 2.7968893105867183 3.271478180632943
3.4801504130242837 1.0000000000000002 4.5259679805866355 4.047864214059725 3.1559934157934872 2.8428228353734415 2.525472490762333 3.1446998835631006 3.4222850690163105 3.3308005033440415 7.25743651677248 9.052263222753588 3.854485119995204 2.352338985977772
3.464338802507882 4.5259679805866355 0.9999999999999998 2.877501458540774 4.573195051671071 44.845194179408196 4.1508693102941585 3.1947995999761214 5.073874573966015 4.217622990945619 6.672993143171266 1.831029857591979 2.242075678844558 2.276384607102156
9.40120014368047 4.047864214059725 2.877501458540774 0.9999999999999996 4.293798273981672 11.894276912140214 8.638319338303766 6.389993571240275 6.072023045743229 6.653257974765394 18.275152792205606 8.203178910357911 4.9592480212

In [13]:
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
import json
elec = json.load(open("../data/review/50EleReviews.json"))

In [4]:
item = 17777
print(elec['target_names'])
print(elec['target_names'][elec['y'][item]])
print(elec['X'][item])
for i, c in enumerate(elec['target_names']):
    print(i+1, c)

['Monitor', 'Watch', 'Blu Ray Player', 'CDPlayer', 'Network Adapter', 'Battery', 'Graphics Card', 'Xbox', 'Telephone', 'Car Stereo', 'Video Recorder', 'Alarm Clock', 'Camera', 'Camcorder', 'HardDrive', 'Home Theater System', 'Subwoofer', 'Rice Cooker', 'Webcam', 'Kindle', 'Scanner', 'Radar Detector', 'Vacuum', 'Cable Modem', 'CellPhone', 'Wireless Router', 'Printer', 'Fan', 'Amplifier', 'Microwave', 'GPS', 'Microphone', 'TV', 'Projector', 'Mouse', 'Voice Recorder', 'Memory Card', 'Iron', 'Laptop', 'Headphone', 'Video Player', 'MP3 Player', 'Keyboard', 'Tablet', 'Media Player', 'Lamp', 'Remote Control', 'Speaker', 'Computer', 'DVDPlayer']
Rice Cooker
Its fantastic ! I was debating on whether to buy a rice cooker or continue buying rice from the Chinese takeout shop downstairs . After reading multiple reviews I finally decided on this one . although I was pre-warned by the reviews that it was going to be a small cooker , I was really surprised when it arrived . I did n't expect it to be 

In [96]:
from nltk.stem import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()

In [32]:
stop_words = set(stopwords.words('english'))

In [102]:
class Category:
    
    def __init__(self, label, description, hierarchy): # Create an empty path
        self.label = label.strip().lower()
        self.description = description.strip().lower()
        self.hierarchy = hierarchy.strip().lower().split(';')
        self.nodes = {'the_class':[],
                      'super_class':[],
                      'description':[],
#                       'synonym':[],
#                       'related_term':[]
                     }
        self.find_nodes()
    
    def __repr__(self):
        return self.label + ' => \n' + '\n'.join([key + ': ' + str(val) for key, val in self.nodes.items()]) + '\n'
    
    def find_nodes(self):
        # The class
        self.nodes['the_class'] = get_all_nodes_from_label(self.label)
                
        # Super class
        for a_super_class in self.hierarchy:
            self.nodes['super_class'].extend(get_all_nodes_from_label(a_super_class))
        self.nodes['super_class'] = list(set(self.nodes['super_class']))
        
        # Description
        text = nltk.pos_tag(word_tokenize(self.description))
#         print(text)
        for token in text:
#             if (token[1].startswith('NN') or token[1].startswith('JJ')) and token[0] not in stop_words: # (Noun or Adjective) and not stop words
#                 lemmatised_token = lemmatizer.lemmatize(token[0], pos="a") if token[1].startswith('JJ') else lemmatizer.lemmatize(token[0], pos="n")
            if token[1].startswith('NN') and token[0] not in stop_words: # Noun and not stop words
                lemmatised_token = lemmatizer.lemmatize(token[0], pos="n")
                self.nodes['description'].extend(get_all_nodes_from_label(lemmatised_token))
        self.nodes['description'] = list(set(self.nodes['description']))
        
        # Synonym
        # Option 1: Synonyms of every class node
#         for node in self.nodes['the_class']:
#             self.nodes['synonym'].extend(list(SYNONYMS.get(node,set())))
#         self.nodes['synonym'] = list(set(self.nodes['synonym']))
        
        # Option 2: Synonyms of the main class node only
#         if standardized_uri('en', self.label) in ALL_NODES:
#             self.nodes['synonym'] = list(SYNONYMS.get(standardized_uri('en', self.label),set()))

    def get_all_nodes(self):
        ans = []
        for key, val in self.nodes.items():
            ans.extend(val)
        return set(ans)

In [38]:
def get_all_nodes_from_label(label):
    ans = []
    if standardized_uri('en', label) in ALL_NODES:
        ans.append(standardized_uri('en', label))
    for token in label.split():
        if token not in stop_words and standardized_uri('en', token) in ALL_NODES and standardized_uri('en', token) not in ans:
            ans.append(standardized_uri('en', token))
    return ans

In [69]:
def get_strong_synonyms():
    synonyms = dict()
    with open('../wordEmbeddings/conceptnet-assertions-en-5.6.0.csv', 'r', encoding = "utf8") as csvfile:
        reader = csv.reader(csvfile, delimiter='\t')
        for line in reader:
            if line[1] == '/r/Synonym':
                details = json.loads(line[4])
                w = details['weight']
                if w > 1.0:
                    sub = remove_word_sense(line[2])
                    obj = remove_word_sense(line[3])
#                     print(sub, obj)
                    if sub in synonyms:
                        synonyms[sub].add(obj)
                    else:
                        synonyms[sub] = set([obj])
                    
                    if obj in synonyms:
                        synonyms[obj].add(sub)
                    else:
                        synonyms[obj] = set([sub])
                       
    return synonyms

In [15]:
def remove_word_sense(sub):
    if sub.count('/') > 3:
        if sub.count('/') > 4:
            print(sub)
            assert False, "URI error (with more than 4 slashes)"
        sub = sub[:sub.rfind('/')]
    return sub

In [16]:
def get_all_nodes(): # get all distinct uri in conceptnet (without part of speech)
    nodes = set()
    with open('../wordEmbeddings/conceptnet-assertions-en-5.6.0.csv', 'r', encoding = "utf8") as csvfile:
        reader = csv.reader(csvfile, delimiter='\t')
        for line in reader:
            sub = remove_word_sense(line[2])
            obj = remove_word_sense(line[3])
            nodes.add(sub)
            nodes.add(obj)
    return nodes

In [8]:
def get_class_info(filename):
    with open(filename, encoding = 'utf-8') as csvfile:
        reader = csv.DictReader(csvfile, delimiter=',')
        ans = [row for row in reader]
        print("No. of classes =", len(ans))
        print("Header =", ans[0].keys())
        return ans

In [17]:
ALL_NODES = get_all_nodes()
print(len(ALL_NODES))

1100497


In [70]:
SYNONYMS = get_strong_synonyms()
print(len(SYNONYMS))

111011


In [103]:
filename = "../data/dbpedia/classLabelsDBpedia.csv"
class_info = get_class_info(filename)
for c in class_info:
    print(Category(c['ClassLabel'], c['ClassDescription'], c['Hierarchy']))

No. of classes = 14
Header = odict_keys(['ClassCode', 'ClassLabel', 'ClassDescription', 'DBpediaManual', 'ConceptNet', 'Count', 'Hierarchy'])
company => 
the_class: ['/c/en/company']
super_class: ['/c/en/agent', '/c/en/organisation']
description: ['/c/en/good', '/c/en/money', '/c/en/organization', '/c/en/service']

educational institution => 
the_class: ['/c/en/educational_institution', '/c/en/educational', '/c/en/institution']
super_class: ['/c/en/agent', '/c/en/organisation']
description: []

artist => 
the_class: ['/c/en/artist']
super_class: ['/c/en/agent', '/c/en/person']
description: ['/c/en/painting', '/c/en/sculpture', '/c/en/someone']

athlete => 
the_class: ['/c/en/athlete']
super_class: ['/c/en/agent', '/c/en/person']
description: ['/c/en/athletics', '/c/en/sport', '/c/en/competition', '/c/en/someone', '/c/en/part']

office holder => 
the_class: ['/c/en/office_holder', '/c/en/office', '/c/en/holder']
super_class: ['/c/en/agent', '/c/en/person']
description: ['/c/en/governmen

In [106]:
filename = "../data/newsgroup/classLabels20news.csv"
class_info = get_class_info(filename)
for c in class_info:
    print(Category(c['ConceptNet'], c['ClassDescription'], c['Hierarchy']))

No. of classes = 20
Header = odict_keys(['ClassCode', 'ClassLabel', 'ConceptNet', 'Count', 'ClassDescription', 'Hierarchy'])
atheism => 
the_class: ['/c/en/atheism']
super_class: ['/c/en/alt']
description: ['/c/en/belief', '/c/en/god', '/c/en/theory']

graphics => 
the_class: ['/c/en/graphics']
super_class: ['/c/en/computer']
description: ['/c/en/picture', '/c/en/computer']

operating system => 
the_class: ['/c/en/operating_system', '/c/en/operating', '/c/en/system']
super_class: ['/c/en/ms', '/c/en/os', '/c/en/computer', '/c/en/windows']
description: ['/c/en/software', '/c/en/computer', '/c/en/part']

ibm => 
the_class: ['/c/en/ibm']
super_class: ['/c/en/system', '/c/en/hardware', '/c/en/pc', '/c/en/computer']
description: []

mac => 
the_class: ['/c/en/mac']
super_class: ['/c/en/system', '/c/en/hardware', '/c/en/computer']
description: []

windows => 
the_class: ['/c/en/windows']
super_class: ['/c/en/x', '/c/en/computer']
description: []

sale => 
the_class: ['/c/en/sale']
super_clas

In [107]:
print(lemmatizer.lemmatize("12345"))
print(lemmatizer.lemmatize("cacti"))
print(lemmatizer.lemmatize("geese"))
print(lemmatizer.lemmatize("rocks", pos="a"))
print(lemmatizer.lemmatize("python"))
print(lemmatizer.lemmatize("better", pos="a"))
print(lemmatizer.lemmatize("best", pos="a"))
print(lemmatizer.lemmatize("run"))
print(lemmatizer.lemmatize("run",'v'))


12345
cactus
goose
rocks
python
good
best
run
run


In [4]:
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))

pos_dict = {'JJ': 'a', 'JJR': 'a', 'JJS': 'a',
           'NN': 'n', 'NNP': 'n', 'NNPS': 'n', 'NNS': 'n',
           'RB': 'r', 'RBR': 'r', 'RBS': 'r',
           'VB': 'v', 'VBD': 'v', 'VBG': 'v', 'VBN': 'v', 'VBP': 'v', 'VBZ': 'v'}

In [5]:
def process_training_data_tfidf(filename):
    count_of_class = dict()
    count_word_of_class = dict()
    with open(filename, 'r', encoding = "utf8") as csvfile:
        reader = csv.DictReader(csvfile, delimiter=',')
        i = 0
        for line in reader:
            class_number = line['class'].strip()
            count_of_class[class_number] = count_of_class.get(class_number, 0) + 1
            
            text = line['text'].lower().strip()
            text = nltk.pos_tag(word_tokenize(text))
            all_keys = set()
            for token in text:
                tag = token[1]
                if tag not in pos_dict:
                    key = token[0]
                else:
                    key = lemmatizer.lemmatize(token[0], pos_dict[tag])
                all_keys.add(key)
            
            if class_number not in count_word_of_class:
                count_word_of_class[class_number] = dict()
            for k in all_keys:
                count_word_of_class[class_number][k] = count_word_of_class[class_number].get(k, 0) + 1
            
            if i%1000 == 0:
                print(i)
            i += 1
    return count_of_class, count_word_of_class

In [22]:
count_of_class, count_word_of_class = process_training_data_tfidf("../data/dbpedia/train.csv")

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
65000
66000
67000
68000
69000
70000
71000
72000
73000
74000
75000
76000
77000
78000
79000
80000
81000
82000
83000
84000
85000
86000
87000
88000
89000
90000
91000
92000
93000
94000
95000
96000
97000
98000
99000
100000
101000
102000
103000
104000
105000
106000
107000
108000
109000
110000
111000
112000
113000
114000
115000
116000
117000
118000
119000
120000
121000
122000
123000
124000
125000
126000
127000
128000
129000
130000
131000
132000
133000
134000
135000
136000
137000
138000
139000
140000
141000
142000
143000
144000
145000
146000
147000
148000
149000
150000
151000
152000
153000
154000
155000
156000
157000
158000


In [23]:
pickle.dump(count_of_class, open("../wordEmbeddings/dbpedia_count_of_class.pickle", "wb"))
pickle.dump(count_word_of_class, open("../wordEmbeddings/dbpedia_count_word_of_class.pickle", "wb"))

In [2]:
count_of_class = pickle.load(open("../wordEmbeddings/dbpedia_count_of_class.pickle", "rb"))
count_word_of_class = pickle.load(open("../wordEmbeddings/dbpedia_count_word_of_class.pickle", "rb"))

In [24]:
print(count_of_class)

{'1': 40000, '2': 40000, '3': 40000, '4': 40000, '5': 40000, '6': 40000, '7': 40000, '8': 40000, '9': 40000, '10': 40000, '11': 40000, '12': 40000, '13': 40000, '14': 40000}


In [8]:
print(count_word_of_class['1'])

{'growth': 1, 'japan': 1, 'since': 1, "'s": 2, 'beverage': 1, 'company': 5, '2004': 1, 'venture': 1, 'and': 6, 'american': 2, 'its': 2, '2011': 1, 'have': 2, 'new': 1, 'found': 2, 'sake': 1, '50': 1, 'make': 1, 'with': 2, 'other': 2, 'spirit': 1, 'ku': 1, 'a': 2, 'be': 6, 'while': 1, '.': 6, 'specialize': 1, 'two': 2, 'privately-held': 1, 'the': 6, '/taéªkuë\x90/': 1, 'an': 3, 'through': 1, 'headquarter': 1, 'that': 1, 'ty': 1, 'city': 2, 'in': 5, 'base': 3, 'product': 1, 'into': 1, 'alcoholic': 1, 'joint': 1, 'york': 1, 'all': 2, 'extend': 1, 'brewery': 1, 'state': 1, '2001': 1, 'form': 1, 'oddlot': 1, 'decade': 1, 'by': 1, '(': 3, 'novel': 1, 'california.oddlot': 1, 'one': 3, 'another': 1, 'pritzker': 1, 'ender': 1, 'financing': 1, 'entertainment': 1, 'more': 1, ')': 3, 'game': 2, 'release': 1, 'work': 1, 'film': 1, 'produce': 1, 'of': 4, 'or': 1, 'for': 3, 'prete': 1, 'del': 1, 'scott': 1, 'producer': 1, 'than': 1, 'card': 1, 'planner': 1, 'wedding': 1, 'production': 1, 'longtime': 

In [26]:
def calculate_tfidf(count_of_class, count_word_of_class):
    tf_of_class = dict()
    all_words = set()
    idf_of_words = dict()
    for key in count_word_of_class:
        tf_of_class[key] = copy.deepcopy(count_word_of_class[key])
        for k in tf_of_class[key]:
            tf_of_class[key][k] = tf_of_class[key][k] / count_of_class[key]
        all_words = all_words.union(set(tf_of_class[key].keys()))
        print("Finish calculating tf of class", key)
    for w in all_words:
        profile = []
        for key in tf_of_class:
            if w in tf_of_class[key]:
                profile.append(tf_of_class[key][w])
            else:
                profile.append(0)
        profile = np.array(profile)/np.sum(np.array(profile))
        idf_of_words[w] = 1 - (entropy(profile) / math.log2(len(count_of_class)))
    print("Finish calculating idf")
    tf_idf_of_class = dict()
    for key in tf_of_class:
        tf_idf_of_class[key] = copy.deepcopy(tf_of_class[key])
        for k in tf_idf_of_class[key]:
            tf_idf_of_class[key][k] = tf_idf_of_class[key][k] * idf_of_words[k] 
        print("Finish calculating tf-idf of class", key)
    return tf_idf_of_class, tf_of_class, idf_of_words

In [27]:
tf_idf_of_class, tf_of_class, idf_of_words = calculate_tfidf(count_of_class, count_word_of_class)

Finish calculating tf of class 1
Finish calculating tf of class 2
Finish calculating tf of class 3
Finish calculating tf of class 4
Finish calculating tf of class 5
Finish calculating tf of class 6
Finish calculating tf of class 7
Finish calculating tf of class 8
Finish calculating tf of class 9
Finish calculating tf of class 10
Finish calculating tf of class 11
Finish calculating tf of class 12
Finish calculating tf of class 13
Finish calculating tf of class 14
Finish calculating idf
Finish calculating tf-idf of class 1
Finish calculating tf-idf of class 2
Finish calculating tf-idf of class 3
Finish calculating tf-idf of class 4
Finish calculating tf-idf of class 5
Finish calculating tf-idf of class 6
Finish calculating tf-idf of class 7
Finish calculating tf-idf of class 8
Finish calculating tf-idf of class 9
Finish calculating tf-idf of class 10
Finish calculating tf-idf of class 11
Finish calculating tf-idf of class 12
Finish calculating tf-idf of class 13
Finish calculating tf-idf

In [30]:
pickle.dump(tf_idf_of_class, open("../wordEmbeddings/dbpedia_tf_idf_of_class.pickle", "wb"))
pickle.dump(tf_of_class, open("../wordEmbeddings/dbpedia_tf_of_class.pickle", "wb"))
pickle.dump(idf_of_words, open("../wordEmbeddings/dbpedia_idf_of_words.pickle", "wb"))

In [38]:
for key in tf_idf_of_class:
#     print(list(tf_idf_of_class[key].items()))
    ar = sorted(list(tf_idf_of_class[key].items()), key=lambda x: x[1], reverse=True)
    print('Class', key)
    for i in range(20):
        print(ar[i][0], ' / ', ar[i][1], ' / ', tf_of_class[key][ar[i][0]], ' / ', idf_of_words[ar[i][0]])
    print('---------------------------------')

Class 1
company  /  0.2995668363373626  /  0.5261  /  0.5694104473243919
found  /  0.12024099455026939  /  0.257675  /  0.46663818589412787
headquarter  /  0.08821475325112814  /  0.101975  /  0.8650625472040024
base  /  0.07233070396973548  /  0.252425  /  0.2865433454282875
service  /  0.07148824520886987  /  0.16665  /  0.42897236849006826
product  /  0.06972228349294496  /  0.0981  /  0.71072664111055
operate  /  0.05969666634817688  /  0.133175  /  0.4482573031588277
manufacturer  /  0.05453036738510578  /  0.07735  /  0.7049821252114515
inc.  /  0.05293762280860346  /  0.086525  /  0.6118188131592425
subsidiary  /  0.04839273471681463  /  0.069925  /  0.6920662812558402
label  /  0.047482115447262184  /  0.077575  /  0.6120801217823033
by  /  0.04585192382889403  /  0.388275  /  0.11809136263960862
software  /  0.04573591871907625  /  0.0548  /  0.8345970569174498
airline  /  0.045448727754575646  /  0.0527  /  0.8624047012253444
record  /  0.040087606164021336  /  0.0802  /  0.4

In [11]:
import math, copy
def entropy(a):
    return np.sum([-x*math.log2(x) for x in a if x!=0])

In [32]:
tfidf_log = pickle.load(open('../wordEmbeddings/TFIDF_class_divided_by_most_freq.pkl','rb'))

In [35]:
print(type(tfidf_log), list(tfidf_log.keys()))
print(tfidf_log[1])

<class 'dict'> [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14]
[ 0.00000000e+00 -3.98948242e-02 -3.30879384e-02 ...  0.00000000e+00
  6.62969403e-06  0.00000000e+00]


In [71]:
def get_seen_unseen(class_names, seen_ratio):
    num_seen = seen_ratio * len(class_names)
    if num_seen - int(num_seen) >= 0.5:
        num_seen = int(num_seen) + 1
    else:
        num_seen = int(num_seen)
        
    all_classes = list(class_names)
    random.shuffle(all_classes)
    seen_classes = all_classes[:num_seen]
    unseen_classes = all_classes[num_seen:]
    return seen_classes, unseen_classes

In [74]:
def create_training_data(count_of_class, count_word_of_class, unseen_classes):
    new_count_of_class = copy.deepcopy(count_of_class)
    new_count_word_of_class = copy.deepcopy(count_word_of_class)
    for key in unseen_classes:
        new_count_of_class.pop(key, None)
        new_count_word_of_class.pop(key, None)
    return calculate_tfidf(new_count_of_class, new_count_word_of_class)

In [72]:
# Compare TF-IDF with seen classes only and with unseen classes
print(get_seen_unseen(list(count_of_class.keys()), seen_ratio = 0.75))

(['7', '1', '11', '14', '13', '5', '8', '10', '12', '4', '2'], ['9', '3', '6'])


In [75]:
seen_classes, unseen_classes = get_seen_unseen(list(count_of_class.keys()), seen_ratio = 0.75)
train_tf_idf, train_tf, train_idf = create_training_data(count_of_class, count_word_of_class, unseen_classes)

Finish calculating tf of class 2
Finish calculating tf of class 4
Finish calculating tf of class 5
Finish calculating tf of class 6
Finish calculating tf of class 8
Finish calculating tf of class 9
Finish calculating tf of class 10
Finish calculating tf of class 11
Finish calculating tf of class 12
Finish calculating tf of class 13
Finish calculating tf of class 14
Finish calculating idf
Finish calculating tf-idf of class 2
Finish calculating tf-idf of class 4
Finish calculating tf-idf of class 5
Finish calculating tf-idf of class 6
Finish calculating tf-idf of class 8
Finish calculating tf-idf of class 9
Finish calculating tf-idf of class 10
Finish calculating tf-idf of class 11
Finish calculating tf-idf of class 12
Finish calculating tf-idf of class 13
Finish calculating tf-idf of class 14


In [6]:
import random
random.sample([1,2,3,4,5],1)
set([1,2,3,4]).difference(set([2]))

{1, 3, 4}